In [4]:
!pip install neo4j~=5.28.0

Defaulting to user installation because normal site-packages is not writeable
  Attempting uninstall: neo4j
    Found existing installation: neo4j 6.0.3
    Uninstalling neo4j-6.0.3:
      Successfully uninstalled neo4j-6.0.3



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
!pip install odfpy

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/717.0 kB ? eta -:--:--
     ---------------------------------------- 0.0/717.0 kB ? eta -:--:--
     -------------- ------------------------- 262.1/717.0 kB ? eta -:--:--
     -------------------------------------- 717.0/717.0 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for odfpy: filename=odfpy-1.4.1-py2.py3-none-any.whl size=137565 sha256=38e54ab21b04c7a8968cace366fb2349c1bbbaa036c92ea18d73442cfd1d52a2
  Stored in directory: c:\users\genaiblrancusr66\appdata\local\pip\cache\wheels\36\5d\63\8243a7ee78fff0f944d638fd0e66d7278888f5e2285d7346b


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [37]:
import pandas as pd
from neo4j import GraphDatabase
import os 
import re

# ====================================================================
# 1. CONFIGURATION
# ====================================================================
URI = "bolt://localhost:7687"
USER = "neo4j"
PASSWORD = "KushalKuldipSuhas" 
GRANT_EXCEL_PATH = "grants.xlsx" 

# ====================================================================
# 2. DATA PREPROCESSING FUNCTION 
# ====================================================================

def load_and_preprocess_data(file_path):
    """
    Loads data from Excel, cleans, and prepares for Neo4j insertion.
    """
    
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Excel file not found at: {file_path}")
    
    # Load Excel file
    df = pd.read_excel(file_path)
    
    print(f"Original columns: {df.columns.tolist()}")
    
    # Clean column names - remove spaces, special chars, lowercase
    # This ensures 'Country' becomes 'Country' (or 'country' if you lowercased it, but here it preserves case)
    df.columns = [re.sub(r'\s+', '', str(col).strip()) for col in df.columns]
    
    print(f"Cleaned columns: {df.columns.tolist()}")
    
    # Column mapping based on your actual Excel structure
    # Added 'Country' mapping here
    COLUMN_MAP = {
        'grantid': 'id',
        'grantname': 'name',
        'fundingtype': 'funding_type', 
        'maximumprojectvalueinr': 'max_value',
        'maxgrantsubsidy': 'max_subsidy',
        'targetverticals': 'verticals_raw',
        'keytechnicalfocus': 'tech_focus_raw',
        'SMESizeEligibility': 'size_eligibility_raw',
        'Must-HaveCriterion1': 'criterion_1',
        'Must-HaveCriterion2': 'criterion_2',
        'AdditionalGeographicFilter': 'geo_filter_raw',
        'Country': 'country_raw',  # <--- NEW MAPPING
    }
    
    # Rename columns
    df = df.rename(columns=COLUMN_MAP)
    
    # Add ID column if not present (using row index)
    if 'id' not in df.columns:
        df['id'] = ['GRANT_' + str(i+1).zfill(3) for i in range(len(df))]
    
    # Function to clean and split multi-value fields
    def clean_and_split(value):
        """Split comma/semicolon separated values, handle N/A and empty values"""
        if pd.isna(value):
            return []
        
        value_str = str(value).strip()
        
        # Handle special cases
        if value_str.lower() in ['n/a', 'none', '']:
            return []
        
        # Split by comma or semicolon
        items = re.split(r'[,;]', value_str)
        
        # Clean each item
        cleaned = []
        for item in items:
            item = item.strip()
            if item and item.lower() not in ['n/a', 'none', '']:
                cleaned.append(item)
        
        return cleaned
    
    # Process relationship fields
    df['verticals'] = df.get('verticals_raw', pd.Series(dtype=object)).apply(clean_and_split)
    df['tech_focus'] = df.get('tech_focus_raw', pd.Series(dtype=object)).apply(clean_and_split)
    df['size_eligibility'] = df.get('size_eligibility_raw', pd.Series(dtype=object)).apply(clean_and_split)
    df['geo_filter'] = df.get('geo_filter_raw', pd.Series(dtype=object)).apply(clean_and_split)
    
    # Process the new Country field (treated as list to handle multiple countries if needed)
    df['country'] = df.get('country_raw', pd.Series(dtype=object)).apply(clean_and_split) # <--- NEW PROCESSING
    
    # Debug: Show what we're getting
    print(f"\nSample verticals: {df['verticals'].head(3).tolist()}")
    print(f"Sample tech_focus: {df['tech_focus'].head(3).tolist()}")
    print(f"Sample country: {df['country'].head(3).tolist()}") # <--- NEW DEBUG PRINT
    
    # Process scalar fields - convert NaN to None
    for col in ['name', 'funding_type', 'max_value', 'max_subsidy', 'criterion_1', 'criterion_2']:
        if col in df.columns:
            df[col] = df[col].apply(lambda x: None if pd.isna(x) else str(x).strip())
    
    # Select final columns
    # Added 'country' to final columns
    final_columns = ['id', 'name', 'funding_type', 'max_value', 'max_subsidy', 
                     'verticals', 'tech_focus', 'size_eligibility', 'geo_filter',
                     'criterion_1', 'criterion_2', 'country']
    
    # Only keep columns that exist
    available_columns = [col for col in final_columns if col in df.columns]
    processed_data = df[available_columns].to_dict('records')
    
    # Final type validation
    for record in processed_data:
        # Ensure lists are lists (Added 'country' here)
        for key in ['verticals', 'tech_focus', 'size_eligibility', 'geo_filter', 'country']:
            if key not in record:
                record[key] = []
            elif not isinstance(record[key], list):
                record[key] = []
        
        # Ensure strings are strings or None
        for key in ['name', 'funding_type', 'max_value', 'max_subsidy', 'criterion_1', 'criterion_2']:
            if key not in record:
                record[key] = None
            elif pd.isna(record.get(key)):
                record[key] = None
    
    print(f"\nProcessed {len(processed_data)} grant records")
    if processed_data:
        print(f"\nSample record:")
        sample = processed_data[0]
        print(f"  ID: {sample.get('id')}")
        print(f"  Name: {sample.get('name')}")
        print(f"  Country: {sample.get('country')}") # <--- NEW PRINT
    
    return processed_data

# ====================================================================
# 3. NEO4J GRAPH CREATION FUNCTION
# ====================================================================

def create_knowledge_graph(uri, user, password, data):
    """
    Creates a knowledge graph in Neo4j with proper hierarchical relationships.
    """
    if not data:
        print("No data to process. Exiting graph creation.")
        return

    driver = None
    try:
        driver = GraphDatabase.driver(uri, auth=(user, password))
        print("Connected to Neo4j successfully.")

        # Clear existing data
        with driver.session() as session:
            print("Clearing existing graph data...")
            session.run("MATCH (n) DETACH DELETE n")

        cypher_query = """
        UNWIND $grant_list AS g
        
        // 1. Create/Update Grant Node
        MERGE (grant:Grant {id: g.id})
        SET grant.name = g.name,
            grant.funding_type = g.funding_type,
            grant.max_value = g.max_value,
            grant.max_subsidy = g.max_subsidy
        
        // 2. Verticals
        FOREACH (vertical_name IN [v IN g.verticals WHERE v IS NOT NULL AND TRIM(v) <> ''] |
            MERGE (vert:Vertical {name: TRIM(vertical_name)})
            MERGE (grant)-[:TARGETS_VERTICAL]->(vert)
        )
        
        // 3. Technologies
        FOREACH (tech_name IN [t IN g.tech_focus WHERE t IS NOT NULL AND TRIM(t) <> ''] |
            MERGE (tech:Technology {name: TRIM(tech_name)})
            MERGE (grant)-[:USES_TECH]->(tech)
        )
        
        // 4. SME Size Eligibility
        FOREACH (size_name IN [s IN g.size_eligibility WHERE s IS NOT NULL AND TRIM(s) <> ''] |
            MERGE (sz:Size {name: TRIM(size_name)})
            MERGE (grant)-[:ELIGIBLE_FOR_SIZE]->(sz)
        )
        
        // 5. Criterion 1
        FOREACH (c1_desc IN CASE WHEN g.criterion_1 IS NOT NULL AND TRIM(g.criterion_1) <> '' 
                                  THEN [g.criterion_1] ELSE [] END |
            MERGE (c1:Criterion {description: TRIM(c1_desc)})
            ON CREATE SET c1.type = 'Must-Have 1'
            MERGE (grant)-[:REQUIRES_CRITERION {type: 'Must-Have 1'}]->(c1)
        )
        
        // 6. Criterion 2
        FOREACH (c2_desc IN CASE WHEN g.criterion_2 IS NOT NULL AND TRIM(g.criterion_2) <> '' 
                                  THEN [g.criterion_2] ELSE [] END |
            MERGE (c2:Criterion {description: TRIM(c2_desc)})
            ON CREATE SET c2.type = 'Must-Have 2'
            MERGE (grant)-[:REQUIRES_CRITERION {type: 'Must-Have 2'}]->(c2)
        )
        
        // 7. Geographic Filters
        FOREACH (region_name IN [r IN g.geo_filter WHERE r IS NOT NULL AND TRIM(r) <> ''] |
            MERGE (reg:Region {name: TRIM(region_name)})
            MERGE (grant)-[:HAS_GEOGRAPHIC_FILTER]->(reg)
        )
        
        // 8. Country (NEW SECTION)
        FOREACH (country_name IN [c IN g.country WHERE c IS NOT NULL AND TRIM(c) <> ''] |
            MERGE (cntry:Country {name: TRIM(country_name)})
            MERGE (grant)-[:APPLICABLE_TO_COUNTRY]->(cntry)
        )
        
        RETURN count(DISTINCT grant) AS processedGrants
        """

        with driver.session() as session:
            result = session.run(cypher_query, grant_list=data)
            summary = result.consume()
            
            print("\n" + "="*60)
            print("✅ Graph Creation Complete")
            print("="*60)
            print(f"Nodes Created: {summary.counters.nodes_created}")
            print(f"Relationships Created: {summary.counters.relationships_created}")
            print(f"Properties Set: {summary.counters.properties_set}")
            print("="*60)
            
            # Verify the graph
            print("\nVerifying graph structure...")
            verify_query = """
            MATCH (g:Grant)
            RETURN 'Grants' as entity, count(g) as count
            UNION ALL
            MATCH (v:Vertical)
            RETURN 'Verticals' as entity, count(v) as count
            UNION ALL
            MATCH (t:Technology)
            RETURN 'Technologies' as entity, count(t) as count
            UNION ALL
            MATCH (s:Size)
            RETURN 'Sizes' as entity, count(s) as count
            UNION ALL
            MATCH (c:Criterion)
            RETURN 'Criteria' as entity, count(c) as count
            UNION ALL
            MATCH (r:Region)
            RETURN 'Regions' as entity, count(r) as count
            UNION ALL
            MATCH (cntry:Country)
            RETURN 'Countries' as entity, count(cntry) as count
            UNION ALL
            MATCH ()-[r:APPLICABLE_TO_COUNTRY]->()
            RETURN 'Country Relationships' as entity, count(r) as count
            """
            
            # Added Verification for other existing relationships (condensed previous verify query for brevity in display)
            # You can keep the full original verification query and just append the Country parts
            
            full_verify_query = """
            MATCH (g:Grant) RETURN 'Grants' as entity, count(g) as count
            UNION ALL MATCH (v:Vertical) RETURN 'Verticals' as entity, count(v) as count
            UNION ALL MATCH (t:Technology) RETURN 'Technologies' as entity, count(t) as count
            UNION ALL MATCH (s:Size) RETURN 'Sizes' as entity, count(s) as count
            UNION ALL MATCH (c:Criterion) RETURN 'Criteria' as entity, count(c) as count
            UNION ALL MATCH (r:Region) RETURN 'Regions' as entity, count(r) as count
            UNION ALL MATCH (cntry:Country) RETURN 'Countries' as entity, count(cntry) as count
            UNION ALL MATCH ()-[r:TARGETS_VERTICAL]->() RETURN 'Vertical Relationships' as entity, count(r) as count
            UNION ALL MATCH ()-[r:USES_TECH]->() RETURN 'Tech Relationships' as entity, count(r) as count
            UNION ALL MATCH ()-[r:ELIGIBLE_FOR_SIZE]->() RETURN 'Size Relationships' as entity, count(r) as count
            UNION ALL MATCH ()-[r:REQUIRES_CRITERION]->() RETURN 'Criterion Relationships' as entity, count(r) as count
            UNION ALL MATCH ()-[r:HAS_GEOGRAPHIC_FILTER]->() RETURN 'Geographic Relationships' as entity, count(r) as count
            UNION ALL MATCH ()-[r:APPLICABLE_TO_COUNTRY]->() RETURN 'Country Relationships' as entity, count(r) as count
            """

            verification = session.run(full_verify_query)
            print("\nGraph Statistics:")
            for record in verification:
                print(f"  {record['entity']:.<30} {record['count']}")

    except Exception as e:
        print(f"\n❌ ERROR during graph creation: {e}")
        import traceback
        traceback.print_exc()
    finally:
        if driver:
            driver.close()
            print("\nNeo4j connection closed.")

# ====================================================================
# 4. EXECUTION
# ====================================================================

if __name__ == "__main__":
    try:
        print("="*60)
        print("Starting Grant Knowledge Graph Pipeline")
        print("="*60 + "\n")
        
        # 1. Load and preprocess the data
        grant_records = load_and_preprocess_data(GRANT_EXCEL_PATH)
        
        # 2. Create the graph in Neo4j
        create_knowledge_graph(URI, USER, PASSWORD, grant_records)
        
        print("\n" + "="*60)
        print("Pipeline completed successfully!")
        print("="*60)

    except FileNotFoundError as e:
        print(f"\n❌ ERROR: {e}")
    except Exception as e:
        print(f"\n❌ An unexpected error occurred: {e}")
        import traceback
        traceback.print_exc()

Starting Grant Knowledge Graph Pipeline

Original columns: ['grantid', 'grantname', 'fundingtype', 'maximumprojectvalueinr', 'maxgrantsubsidy', 'targetverticals', 'keytechnicalfocus', 'SME Size Eligibility', 'Must-Have Criterion 1', 'Must-Have Criterion 2', 'Additional Geographic Filter', 'Country']
Cleaned columns: ['grantid', 'grantname', 'fundingtype', 'maximumprojectvalueinr', 'maxgrantsubsidy', 'targetverticals', 'keytechnicalfocus', 'SMESizeEligibility', 'Must-HaveCriterion1', 'Must-HaveCriterion2', 'AdditionalGeographicFilter', 'Country']

Sample verticals: [['Manufacturing', 'Service'], ['Manufacturing (High-Energy Clusters)'], ['All Verticals (Commercial/Industrial)']]
Sample tech_focus: [['All Green Technologies (EE', 'RE', 'WtE', 'Green Building)'], ['Energy Efficiency (EE) Upgrades (e.g.', 'Motors', 'VFDs)'], ['Rooftop Solar PV ($\\le$ 500 kWp)']]
Sample country: [['India'], ['India'], ['India']]

Processed 11 grant records

Sample record:
  ID: GIFT-001
  Name: MSE-GIFT (I